# Time Series Forecasting with a Decision Tree

## getting the data

In [1]:
import pandas as pd
with open("../data/current_measurements.json") as file:
    df = pd.read_json(file)

## preparing a training and test set

In [2]:
df = df.iloc[::-1] # reverse the order, the data was ordered by DATE DESC, but we want the more recent data to be used in the test set
df['measuredAt'] = pd.to_datetime(df['measuredAt'])
df.index = df['measuredAt']
del(df['measuredAt'])

test_size = 0.2
lookback = 24

import numpy as np
dataset = df.humidity.values

X, y = [], []
for i in range(lookback, len(dataset) - lookback):
    X.append(dataset[i-lookback:i])
    y.append(dataset[i:i+lookback])
    
# split into train and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, shuffle=False)

## Training the Decision Tree Regressor

In [3]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()
tree_reg.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')